# Modeling

## Load Packages

In [2]:
# TODO: add annotations describing usage of different modules

from operator import mod
from os import getcwd
from os.path import exists, join

import joblib
from sklearn.datasets import fetch_california_housing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVR
import pandas as pd
import numpy as np
# from ydata_profiling import ProfileReport

from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.linear_model import LogisticRegression, LinearRegression
import warnings
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import  GradientBoostingClassifier
# import xgboost as xgb
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC, LinearSVC 
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold

from sklearn.metrics import recall_score

from sklearn import tree
from sklearn.decomposition import PCA, SparsePCA

from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
import json
import pickle
from IPython.display import Image
import warnings

from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from collections import Counter

import altair as alt
import random
import warnings

warnings.filterwarnings('ignore')

## Load Data

In [3]:
cdc_survey = pd.read_csv('../data/cdc_nhanes_survey_responses_clean.csv')

# filter to pregnant moms
cdc_survey_pmom = cdc_survey[cdc_survey['has_been_pregnant'] == 1]
cdc_survey_pmom

,SEQN,SMQ681,SMQ690A,SMQ710,SMQ720,SMQ725,SMQ690B,SMQ740,SMQ690C,SMQ770,...,live_birth_count,age_at_first_birth,age_at_last_birth,months_since_birth,horomones_not_bc,smoked_100_cigs,currently_smoke,height_in,weight_lbs,attempt_weight_loss_1yr
8,109284,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,17.0,24.0,NaN,2.0,2.0,NaN,60.0,178.0,1.0
11,109290,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,2.0,2.0,NaN,63.0,155.0,1.0
12,109291,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,21.0,39.0,NaN,2.0,2.0,NaN,64.0,148.0,2.0
15,109295,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,18.0,24.0,NaN,2.0,2.0,NaN,63.0,137.0,2.0
18,109300,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,30.0,33.0,NaN,2.0,2.0,NaN,59.0,130.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29140,83700,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,27.0,NaN,2.0,2.0,NaN,64.0,153.0,2.0
29141,83701,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,26.0,NaN,2.0,2.0,NaN,65.0,160.0,2.0
29144,83711,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,27.0,30.0,NaN,2.0,2.0,NaN,61.0,185.0,NaN
29146,83717,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,19.0,40.0,NaN,2.0,2.0,NaN,NaN,93.0,2.0


In [4]:
cdc_survey_women = cdc_survey[cdc_survey["is_male"]==0]
cdc_survey_women

,SEQN,SMQ681,SMQ690A,SMQ710,SMQ720,SMQ725,SMQ690B,SMQ740,SMQ690C,SMQ770,...,live_birth_count,age_at_first_birth,age_at_last_birth,months_since_birth,horomones_not_bc,smoked_100_cigs,currently_smoke,height_in,weight_lbs,attempt_weight_loss_1yr
0,109264,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,109266,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,2.0,NaN,64.0,210.0,1.0
5,109277,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,109279,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.0,122.0,2.0
8,109284,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,17.0,24.0,NaN,2.0,2.0,NaN,60.0,178.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35651,83645,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35673,83681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35685,83706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35689,83710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
np.mean(cdc_survey['MDD'])

0.04234582423122164

In [6]:
np.mean(cdc_survey_pmom['MDD'])

0.0971450716961633

In [7]:
np.mean(cdc_survey_women['MDD'])

0.056716417910447764

In [8]:
cdc_survey.groupby(['folder'])['MDD'].sum()

folder
2013_2014         404
2015_2016         392
2017_march2020    716
Name: MDD, dtype: int64

In [9]:
cdc_survey_pmom.groupby(['folder'])['MDD'].sum()

folder
2013_2014         204
2015_2016         200
2017_march2020    348
Name: MDD, dtype: int64

### Features to Use

In [10]:
all_columns = [
    # Depression screener
    'little_interest_in_doing_things',
    'feeling_down_depressed_hopeless',
    'trouble_falling_or_staying_asleep',
    'feeling_tired_or_having_little_energy',
    'poor_appetitie_or_overeating',
    'feeling_bad_about_yourself',
    'trouble_concentrating',
    'moving_or_speaking_to_slowly_or_fast',
    'thoughts_you_would_be_better_off_dead',
    'difficult_doing_daytoday_tasks',
    # Alcohol & smoking
    'has_smoked_tabacco_last_5days',
    'alcoholic_drinks_past_12mo', 
    'drank_alc',
    'alc_drinking_freq',
    'alc_per_day',
    'times_with_4or5_alc',
    'times_with_8plus_alc',
    'times_with_12plus_alc',
    '4plus_alc_daily',
    'days_4plus_drinks_occasion',
    #Blood Pressure & Cholesterol
    'high_bp',
    'age_hypertension',
    'hypertension_prescription',
    'high_bp_prescription',
    'high_cholesterol',
    'cholesterol_prescription',
    #Cardiovascular Health
    'chest_discomfort',
    # Diet & Nutrition
    'how_healthy_is_your_diet',    
    'count_lost_10plus_pounds',
    'has_tried_to_lose_weight_12mo', 
    'breastfed',
    'milk_consumption_freq',
    'govmnt_meal_delivery',
    'nonhomemade_meals',
    'fastfood_meals',
    'readytoeat_meals',
    'frozen_pizza',
    #Food Security
    'emergency_food_received',
    'food_stamps_used',
    'wic_benefit_used',
    #Hospital Utilization & Access to Care
    'general_health',
    'regular_healthcare_place',
    'time_since_last_healthcare',
    'overnight_in_hospital',
    'seen_mental_health_professional',
    #Health Insurance
    'have_health_insurance',
    'have_private_insurance',
    'plan_cover_prescriptions',
    #Income
    'family_poverty_level',
    'family_poverty_level_category',
    #Medical Conditions
    'asthma',
    'anemia_treatment',
    'blood_transfusion',
    'arthritis',
    'heart_failure',
    'coronary_heart_disease',
    'angina_pectoris',
    'heart_attack',
    'stroke',
    'thyroid_issues',
    'respiratory_issues',
    'abdominal_pain',
    'gallstones',
    'gallbladder_surgery',
    'cancer',
    'dr_recommend_lose_weight',
    'dr_recommend_exercise',
    'dr_recommend_reduce_salt',
    'dr_recommend_reduce_fat',
    'currently_losing_weight',
    'currently_increase_exercise',
    'currently_reducing_salt',
    'currently_reducing_fat',
    'metal_objects',
    #Occupation
    'hours_worked',
    'over_35_hrs_worked',
    'work_schedule',
    #Physical Activity
    'vigorous_work',
    'walk_or_bicycle',
    'vigorous_recreation',
    'moderate_recreation',
    # Physical health & Medical History
    'count_days_seen_doctor_12mo',
    'duration_last_healthcare_visit',        
    'count_days_moderate_recreational_activity',   
    'count_minutes_moderate_recreational_activity',
    'count_minutes_moderate_sedentary_activity',
    'general_health_condition',    
    'has_diabetes',
    'has_overweight_diagnosis',  
    #Reproductive Health
    'regular_periods',
    'age_last_period',
    'try_pregnancy_1yr',
    'see_dr_fertility',
    'pelvic_infection',
    'pregnant_now',
    'pregnancy_count',
    'diabetes_pregnancy',
    'delivery_count',
    'live_birth_count',
    'age_at_first_birth',
    'age_at_last_birth',
    'months_since_birth',
    'horomones_not_bc',
    #Smoking
    'smoked_100_cigs',
    'currently_smoke',
    #Weight History
    'height_in',
    'weight_lbs',
    'attempt_weight_loss_1yr',
    # Demographic data
    'food_security_level_household',   
    'food_security_level_adult',    
    'monthly_poverty_index_category',
    'monthly_poverty_index',
    'count_hours_worked_last_week',
    'age_in_years',   
    'education_level',
    'is_usa_born',    
    'has_health_insurance',
    'has_health_insurance_gap'   
]
len(all_columns)

118

In [11]:
# find columns where everything is null
# need to delete those
#cdc_survey_pmom = cdc_survey_pmom[all_columns]
#columns = cdc_survey_pmom.columns[cdc_survey_pmom.notnull().any() == False]
#columns

## Preprocessing analysis

### Remove null values

In [52]:
def get_model_data(original_df, 
                   columns, 
                   test_size_to_use=0.2,
                   drop_null_rows=False,
                   null_imputer_strategy='median', # mean, median, most_frequent
                   use_value_scaler=True,
                   use_smote=False):
    """
    Function to build feature & indicator matrices for both train & test.
    """
    
    # add target column (MDD)
    cols_to_use = columns.copy()
    cols_to_use.insert(0, 'MDD')
    
    df_to_use = original_df[cols_to_use]
    
    if drop_null_rows:
        df_to_use.dropna(inplace=True)
    
    # Create test & train data
    x = df_to_use.iloc[:,1:].values
    y = df_to_use['MDD'].values
    
    if not drop_null_rows:
        # SimpleImputer() = fill in missing values
        # note imputer may drop columns if no values exist for it
        imputer = SimpleImputer(strategy=null_imputer_strategy)  
        x = imputer.fit_transform(x)

    # RobustScaler() = scale features to remove outliers
    if use_value_scaler:
        trans = RobustScaler()
        x = trans.fit_transform(x)

    x_train, x_test, y_train, y_test = train_test_split(
        x, 
        y, 
        test_size=test_size_to_use, 
        random_state=42
    ) 
    
    # Technique to de-risk from positive class imbalance
    if use_smote:
        sm = SMOTE(random_state=42)
        x_train, y_train = sm.fit_resample(x_train, y_train)
    
    return x_train, x_test, y_train, y_test

In [53]:
null_df = cdc_survey_pmom.isnull().sum().to_frame('nulls')
null_df.reset_index(inplace=True)
null_df.columns = ['column', 'nulls']


null_results = null_df[null_df['column'].isin(top_feat)]
null_results['percent null'] = null_results['nulls']/7741
null_results.sort_values('percent null', ascending = False)

,column,nulls,percent null
783,times_with_12plus_alc,7582,0.979460
805,time_since_last_healthcare,7406,0.956724
754,duration_last_healthcare_visit,7406,0.956724
782,times_with_8plus_alc,6973,0.900788
744,count_days_seen_doctor_12mo,6858,0.885932
839,work_schedule,6025,0.778323
749,count_lost_10plus_pounds,4430,0.572277
829,dr_recommend_exercise,4332,0.559618
828,dr_recommend_lose_weight,4331,0.559488
778,drank_alc,4329,0.559230


## Modeling functions

In [14]:
def plot_confusion_matrix(y_test, pred_labels):
    """
    Function that displays a confusion matrix for provided true and predicted classes
    """
    #print(f'cover type 1 and type 2 total correct {np.sum(np.diag(metrics.confusion_matrix(y_test, pred_labels))[:2])}')

    cm = confusion_matrix(y_test, pred_labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    fig, ax = plt.subplots(figsize=(5,5))
    disp = disp.plot(include_values=True, cmap='viridis', ax=ax, xticks_rotation='horizontal')    
    plt.grid(False)
    plt.show()
    return

def get_performance_df(label_actual, label_pred, model_name):
    """
    Function to calculate performance metrics for model.
    Includes precision, recal, F1, & support.
    """
    # create classification report
    result_table = classification_report(label_actual, label_pred, output_dict=True)
    result_table = pd.DataFrame.from_dict(result_table)

    # store for later
    accuracies = result_table['accuracy']

    # rename grouping
    result_table.columns = [
        'depressed_no',
        'depressed_yes',
        'accuracy',
        'macro_avg',
        'weighted_avg'
    ]

    # create dataframe with 1 row per grouping
    result_table.drop(labels = 'accuracy', axis = 1, inplace=True)
    result_table = result_table.transpose()
    result_table['accuracy'] = list(accuracies)
    result_table = result_table.reset_index()
    result_table.rename(columns = {'index':'grouping'},inplace=True)
    result_table['model'] = model_name
    result_table = result_table[['model','grouping','precision','recall','f1-score','support','accuracy']]
    return result_table

def baseline_models(
    x_train, 
    y_train, 
    x_test, 
    y_test,
    do_smote=True,
    show_confusion_matrix=False,
    show_score_dataframe=False,
    show_all_groupings=False,
    return_predictions=False):
    """
    Function that trains and makes predictions using 5 of the classifiers went over during the class.
    Meant as a helper function for easier testing of different modeling pipelines.
    """

    #  do_smote
    if do_smote == True:
        sm = SMOTE(random_state=42)
        x_train, y_train = sm.fit_resample(x_train, y_train)

    # K-Nearest Neighbors
    knn = KNeighborsClassifier()
    knn.fit(x_train, y_train)
    pred_labels_knn  = knn.predict(x_test)
    score_knn = get_performance_df(y_test, pred_labels_knn,'Knn')
    
    # Logistic Regression
    lm = LogisticRegression()
    lm.fit(x_train, y_train)
    pred_labels_lr  = lm.predict(x_test)
    score_lr = get_performance_df(y_test, pred_labels_lr,'Logistic Regression')
        
    # Bernoulii Naive Bayes
    bnb = BernoulliNB()
    bnb.fit(x_train, y_train)
    pred_labels_bnb  = bnb.predict(x_test)
    score_bnb = get_performance_df(y_test, pred_labels_bnb,'Bernoulli Naive Bayes')    
        
    # Gaussian Naive Bayes
    gnb = GaussianNB()
    gnb.fit(x_train, y_train)
    pred_labels_gnb  = gnb.predict(x_test)
    score_gnb = get_performance_df(y_test, pred_labels_gnb,'Gaussian Naive Bayes')    

    # Random Forest
    rf = RandomForestClassifier(random_state=0)
    rf.fit(x_train, y_train)
    pred_labels_rf  = rf.predict(x_test)
    predictions_posterior_rf = rf.predict_proba(x_test)
    score_rf = get_performance_df(y_test, pred_labels_rf,'Random Forest')   
    
    #Decision Tree
    dt = DecisionTreeClassifier()
    dt.fit(x_train, y_train)
    pred_labels_dt = dt.predict(x_test)
    score_dt = get_performance_df(y_test, pred_labels_dt,'Decision Tree')

    #Gradient Boosting Classifier
    gb = GradientBoostingClassifier()
    gb.fit(x_train, y_train)
    pred_labels_gb = gb.predict(x_test)
    score_gb = get_performance_df(y_test, pred_labels_gb,'Gradient Boosting Classifier')
    
#     #XGBoost
#     xgboost = xgb.XGBClassifier()
#     xgboost.fit(x_train, y_train)
#     pred_labels_xgboost = gbt.predict(x_test)
#     score_xgboost = get_performance_df(y_test, pred_labels_xgboost,'XGBoost')
    
    # make dataframe with scores
    scores = pd.concat([score_knn, score_lr, score_bnb, score_gnb, score_rf, score_dt, score_gb])
    scores = scores.sort_values(by = 'recall', ascending=False)
    
    if show_score_dataframe:
        display(scores.style.set_table_attributes('style="font-size: 17px"').hide_index())
    
    if show_confusion_matrix:
        print('\nK-Nearest Neighbors Confusion Matrix')
        plot_confusion_matrix(y_test, pred_labels_knn)
        print('Logistic Regression Confusion Matrix')
        plot_confusion_matrix(y_test, pred_labels_lr)
        print('Bernoulli Naive Bayes Confusion Matrix')
        plot_confusion_matrix(y_test, pred_labels_bnb)
        print('Gaussian Naive Bayes Confusion Matrix')
        plot_confusion_matrix(y_test, pred_labels_gnb)
        print('Random Forest Confusion Matrix')
        plot_confusion_matrix(y_test, pred_labels_rf)
        print('Decision Tree Confusion Matrix')
        plot_confusion_matrix(y_test, pred_labels_dt)
        print('Gradient Boosting Confusion Matrix')
        plot_confusion_matrix(y_test, pred_labels_gbt)
#         print('XGBoost Confusion Matrix')
#         plot_confusion_matrix(y_test, pred_labels_xgboost)

    if not show_all_groupings:
        scores = scores[scores['grouping'] == 'macro_avg']
        
    # make dataframe with predictions
    predictions = pd.DataFrame({
        'actuals':y_test,
        'pred_knn':pred_labels_knn,
        'pred_logistic_regression':pred_labels_lr,
        'pred_bernoulli_naive_bayes':pred_labels_bnb,
        'pred_gaussian_naive_bayes':pred_labels_gnb,
        'pred_random_forest':pred_labels_rf,
        'pred_decision_tree':pred_labels_dt,
        'pred_gradient_boosting_classifier':pred_labels_gb
    })

    if return_predictions:
        return scores, predictions
    else:
        return scores

### Women who have been pregnant, depression screening features only

Evaluate the impact of implementing SMOTE for addressing class imbalances, dropping nulls, incorporating additional features, and filtering on performance

In [18]:
x_train, x_test, y_train, y_test = get_model_data(
    original_df = cdc_survey_pmom,
    columns = all_columns[0:10] # just depression screener
)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(6192, 10)
(1549, 10)
(6192,)
(1549,)


In [22]:
# depression screener only, no dropping nulls, no SMOTE
x_train, x_test, y_train, y_test = get_model_data(
       original_df=cdc_survey_pmom,
       columns = all_columns[0:10],
       test_size_to_use=0.2,
       drop_null_rows=False,
       null_imputer_strategy='median', # mean, median, most_frequent
       use_value_scaler=True,
       use_smote=False)

baseline_model = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
baseline_model_results = baseline_model[['model','precision','recall','f1-score']]
baseline_model_results['features'] = 'depression only'
baseline_model_results['nulls dropped'] = 'False'
baseline_model_results['SMOTE'] = 'False'

baseline_model = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
baseline_model_results = baseline_model[['model','precision','recall','f1-score']]
baseline_model_results['features'] = 'depression only'
baseline_model_results['nulls dropped'] = 'False'
baseline_model_results['SMOTE'] = 'False'

baseline_model_with_smote = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
baseline_model_with_smote_results = baseline_model_with_smote[['model','precision','recall','f1-score']]

baseline_model_with_smote_results['features'] = 'depression only'
baseline_model_with_smote_results['nulls dropped'] = 'False'
baseline_model_with_smote_results['SMOTE'] = 'True'


# depression screener only, dropping nulls, no SMOTE
x_train, x_test, y_train, y_test = get_model_data(
       original_df=cdc_survey_pmom,
       columns = all_columns[0:10],
       test_size_to_use=0.2,
       drop_null_rows=True,
       null_imputer_strategy='median', # mean, median, most_frequent
       use_value_scaler=True,
       use_smote=False)

baseline_model_dropnull = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
baseline_model_dropnull_results= baseline_model_dropnull[['model','precision','recall','f1-score']]

baseline_model_dropnull_results['features'] = 'depression only'
baseline_model_dropnull_results['nulls dropped'] = 'True'
baseline_model_dropnull_results['SMOTE'] = 'False'

baseline_model_dropnull_SMOTE = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
baseline_model_dropnull_SMOTE_results = baseline_model_dropnull_SMOTE[['model','precision','recall','f1-score']]

baseline_model_dropnull_SMOTE_results['features'] = 'depression only'
baseline_model_dropnull_SMOTE_results['nulls dropped'] = 'True'
baseline_model_dropnull_SMOTE_results['SMOTE'] = 'True'


### All women, depression screening features only

In [24]:
# depression screener only, no dropping nulls, no SMOTE
x_train, x_test, y_train, y_test = get_model_data(
       original_df=cdc_survey_women,
       columns = all_columns[0:10],
       test_size_to_use=0.2,
       drop_null_rows=False,
       null_imputer_strategy='median', # mean, median, most_frequent
       use_value_scaler=True,
       use_smote=False)

baseline_model = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
baseline_model_results_women = baseline_model[['model','precision','recall','f1-score']]
baseline_model_results_women['features'] = 'depression only'
baseline_model_results_women['nulls dropped'] = 'False'
baseline_model_results_women['SMOTE'] = 'False'

baseline_model_with_smote = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
baseline_model_with_smote_results_women = baseline_model_with_smote[['model','precision','recall','f1-score']]

baseline_model_with_smote_results_women['features'] = 'depression only'
baseline_model_with_smote_results_women['nulls dropped'] = 'False'
baseline_model_with_smote_results_women['SMOTE'] = 'True'

# depression screener only, dropping nulls, no SMOTE
x_train, x_test, y_train, y_test = get_model_data(
       original_df=cdc_survey_women,
       columns = all_columns[0:10],
       test_size_to_use=0.2,
       drop_null_rows=True,
       null_imputer_strategy='median', # mean, median, most_frequent
       use_value_scaler=True,
       use_smote=False)

baseline_model_dropnull = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
baseline_model_dropnull_results_women= baseline_model_dropnull[['model','precision','recall','f1-score']]

baseline_model_dropnull_results_women['features'] = 'depression only'
baseline_model_dropnull_results_women['nulls dropped'] = 'True'
baseline_model_dropnull_results_women['SMOTE'] = 'False'

baseline_model_dropnull_SMOTE = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
baseline_model_dropnull_SMOTE_results_women = baseline_model_dropnull_SMOTE[['model','precision','recall','f1-score']]

baseline_model_dropnull_SMOTE_results_women['features'] = 'depression only'
baseline_model_dropnull_SMOTE_results_women['nulls dropped'] = 'True'
baseline_model_dropnull_SMOTE_results_women['SMOTE'] = 'True'

### Top identified features

Using the top features only performs better with baseline models compared to using depression screener only features. 
Including SMOTE and/or dropping null values decreases performance.

In [26]:
top_feat = ['times_with_12plus_alc',
 'seen_mental_health_professional',
 'count_days_seen_doctor_12mo',
 'count_lost_10plus_pounds',
 'arthritis',
 'horomones_not_bc',
 'is_usa_born',
 'times_with_8plus_alc',
 'time_since_last_healthcare',
 'duration_last_healthcare_visit',
 'work_schedule',
 'age_in_years',
 'regular_periods',
 'count_minutes_moderate_sedentary_activity',
 'emergency_food_received',
 'high_bp',
 'dr_recommend_exercise',
 'metal_objects',
 'drank_alc',
 'cholesterol_prescription',
 'smoked_100_cigs',
 'vigorous_recreation',
 'dr_recommend_lose_weight',
 'cancer',
 'chest_discomfort',
 'has_health_insurance',
 'have_health_insurance',
 'weight_lbs',
 'high_cholesterol']


high_null_feat = ['times_with_12plus_alc',
                 'duration_last_healthcare_visit',
                 'time_since_last_healthcare',
                 'count_days_seen_doctor_12mo']


top_feat_drop = [feat for feat in top_feat if feat not in high_null_feat]

### Pregnant women, top features only

In [28]:
# top feat only, no dropping nulls, no SMOTE

x_train, x_test, y_train, y_test = get_model_data(
       original_df=cdc_survey_pmom,
       columns = top_feat_drop,
       test_size_to_use=0.2,
       drop_null_rows=False,
       null_imputer_strategy='median', # mean, median, most_frequent
       use_value_scaler=True,
       use_smote=False)

baseline_model = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
baseline_model_top_results = baseline_model[['model','precision','recall','f1-score']]

baseline_model_top_results['features'] = 'top feat'
baseline_model_top_results['nulls dropped'] = 'False'
baseline_model_top_results['SMOTE'] = 'False'


baseline_model_with_smote = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
baseline_model_with_smote_top_results = baseline_model_with_smote[['model','precision','recall','f1-score']]

baseline_model_with_smote_top_results['features'] = 'top feat'
baseline_model_with_smote_top_results['nulls dropped'] = 'False'
baseline_model_with_smote_top_results['SMOTE'] = 'True'

# top feat only, dropping nulls, no SMOTE

x_train, x_test, y_train, y_test = get_model_data(
       original_df=cdc_survey_pmom,
       columns = top_feat_drop,
       test_size_to_use=0.2,
       drop_null_rows=True,
       null_imputer_strategy='median', # mean, median, most_frequent
       use_value_scaler=True,
       use_smote=False)

baseline_model_dropnull = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
baseline_model_dropnull_top_results = baseline_model_dropnull[['model','precision','recall','f1-score']]

baseline_model_dropnull_top_results['features'] = 'top feat'
baseline_model_dropnull_top_results['nulls dropped'] = 'True'
baseline_model_dropnull_top_results['SMOTE'] = 'False'


baseline_model_dropnull_SMOTE = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
baseline_model_dropnull_SMOTE_top_results = baseline_model_dropnull_SMOTE[['model','precision','recall','f1-score']]

baseline_model_dropnull_SMOTE_top_results['features'] = 'top feat'
baseline_model_dropnull_SMOTE_top_results['nulls dropped'] = 'True'
baseline_model_dropnull_SMOTE_top_results['SMOTE'] = 'True'

### All women, top features only

In [29]:
# top feat only, no dropping nulls, no SMOTE

x_train, x_test, y_train, y_test = get_model_data(
       original_df=cdc_survey_women,
       columns = top_feat_drop,
       test_size_to_use=0.2,
       drop_null_rows=False,
       null_imputer_strategy='median', # mean, median, most_frequent
       use_value_scaler=True,
       use_smote=False)

baseline_model = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
baseline_model_top_results_women = baseline_model[['model','precision','recall','f1-score']]

baseline_model_top_results_women['features'] = 'top feat'
baseline_model_top_results_women['nulls dropped'] = 'False'
baseline_model_top_results_women['SMOTE'] = 'False'


baseline_model_with_smote = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
baseline_model_with_smote_top_results_women = baseline_model_with_smote[['model','precision','recall','f1-score']]

baseline_model_with_smote_top_results_women['features'] = 'top feat'
baseline_model_with_smote_top_results_women['nulls dropped'] = 'False'
baseline_model_with_smote_top_results_women['SMOTE'] = 'True'


# top feat only, dropping nulls, no SMOTE

x_train, x_test, y_train, y_test = get_model_data(
       original_df=cdc_survey_women,
       columns = top_feat_drop,
       test_size_to_use=0.2,
       drop_null_rows=True,
       null_imputer_strategy='median', # mean, median, most_frequent
       use_value_scaler=True,
       use_smote=False)


baseline_model_dropnull = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
baseline_model_dropnull_top_results_women = baseline_model_dropnull[['model','precision','recall','f1-score']]

baseline_model_dropnull_top_results_women['features'] = 'top feat'
baseline_model_dropnull_top_results_women['nulls dropped'] = 'True'
baseline_model_dropnull_top_results_women['SMOTE'] = 'False'



baseline_model_dropnull_SMOTE = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
baseline_model_dropnull_SMOTE_top_results_women = baseline_model_dropnull_SMOTE[['model','precision','recall','f1-score']]

baseline_model_dropnull_SMOTE_top_results_women['features'] = 'top feat'
baseline_model_dropnull_SMOTE_top_results_women['nulls dropped'] = 'True'
baseline_model_dropnull_SMOTE_top_results_women['SMOTE'] = 'True'

### Top feat + depression

### Pregnant women, top features only

In [30]:
combined_feat = top_feat_drop + all_columns[0:10]

In [32]:
# combined feat, no dropping nulls, no SMOTE

x_train, x_test, y_train, y_test = get_model_data(
       original_df=cdc_survey_pmom,
       columns = combined_feat,
       test_size_to_use=0.2,
       drop_null_rows=False,
       null_imputer_strategy='median', # mean, median, most_frequent
       use_value_scaler=True,
       use_smote=False)


baseline_model = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
baseline_model_combine_results = baseline_model[['model','precision','recall','f1-score']]

baseline_model_combine_results['features'] = 'top feat+depression'
baseline_model_combine_results['nulls dropped'] = 'False'
baseline_model_combine_results['SMOTE'] = 'False'


baseline_model_with_smote = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
baseline_model_with_smote_combine_results = baseline_model_with_smote[['model','precision','recall','f1-score']]

baseline_model_with_smote_combine_results['features'] = 'top feat+depression'
baseline_model_with_smote_combine_results['nulls dropped'] = 'False'
baseline_model_with_smote_combine_results['SMOTE'] = 'True'

# combined only, dropping nulls, no SMOTE

x_train, x_test, y_train, y_test = get_model_data(
       original_df=cdc_survey_pmom,
       columns = combined_feat,
       test_size_to_use=0.2,
       drop_null_rows=True,
       null_imputer_strategy='median', # mean, median, most_frequent
       use_value_scaler=True,
       use_smote=False)

baseline_model_dropnull = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
baseline_model_dropnull_combine_results = baseline_model_dropnull[['model','precision','recall','f1-score']]

baseline_model_dropnull_combine_results['features'] = 'top feat+depression'
baseline_model_dropnull_combine_results['nulls dropped'] = 'True'
baseline_model_dropnull_combine_results['SMOTE'] = 'False'



baseline_model_dropnull_SMOTE = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
baseline_model_dropnull_SMOTE_combine_results = baseline_model_dropnull_SMOTE[['model','precision','recall','f1-score']]

baseline_model_dropnull_SMOTE_combine_results['features'] = 'top feat+depression'
baseline_model_dropnull_SMOTE_combine_results['nulls dropped'] = 'True'
baseline_model_dropnull_SMOTE_combine_results['SMOTE'] = 'True'

### All women, top features only

In [33]:
# combined feat, no dropping nulls, no SMOTE

x_train, x_test, y_train, y_test = get_model_data(
       original_df=cdc_survey_women,
       columns = combined_feat,
       test_size_to_use=0.2,
       drop_null_rows=False,
       null_imputer_strategy='median', # mean, median, most_frequent
       use_value_scaler=True,
       use_smote=False)


baseline_model = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
baseline_model_combine_results_women = baseline_model[['model','precision','recall','f1-score']]

baseline_model_combine_results_women['features'] = 'top feat+depression'
baseline_model_combine_results_women['nulls dropped'] = 'False'
baseline_model_combine_results_women['SMOTE'] = 'False'


baseline_model_with_smote = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
baseline_model_with_smote_combine_results_women = baseline_model_with_smote[['model','precision','recall','f1-score']]

baseline_model_with_smote_combine_results_women['features'] = 'top feat+depression'
baseline_model_with_smote_combine_results_women['nulls dropped'] = 'False'
baseline_model_with_smote_combine_results_women['SMOTE'] = 'True'


# combined only, dropping nulls, no SMOTE

x_train, x_test, y_train, y_test = get_model_data(
       original_df=cdc_survey_women,
       columns = combined_feat,
       test_size_to_use=0.2,
       drop_null_rows=True,
       null_imputer_strategy='median', # mean, median, most_frequent
       use_value_scaler=True,
       use_smote=False)


baseline_model_dropnull = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
baseline_model_dropnull_combine_results_women = baseline_model_dropnull[['model','precision','recall','f1-score']]

baseline_model_dropnull_combine_results_women['features'] = 'top feat+depression'
baseline_model_dropnull_combine_results_women['nulls dropped'] = 'True'
baseline_model_dropnull_combine_results_women['SMOTE'] = 'False'


baseline_model_dropnull_SMOTE = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
baseline_model_dropnull_SMOTE_combine_results_women = baseline_model_dropnull_SMOTE[['model','precision','recall','f1-score']]

baseline_model_dropnull_SMOTE_combine_results_women['features'] = 'top feat+depression'
baseline_model_dropnull_SMOTE_combine_results_women['nulls dropped'] = 'True'
baseline_model_dropnull_SMOTE_combine_results_women['SMOTE'] = 'True'

In [34]:
#Pregnant women
merge_results=pd.concat([baseline_model_results,
                        baseline_model_with_smote_results,
                        baseline_model_dropnull_results,
                        baseline_model_dropnull_SMOTE_results,
                        baseline_model_top_results,
                        baseline_model_with_smote_top_results,
                        baseline_model_dropnull_top_results,
                        baseline_model_dropnull_SMOTE_top_results,
                        baseline_model_combine_results,
                        baseline_model_with_smote_combine_results,
                        baseline_model_dropnull_combine_results,
                        baseline_model_dropnull_SMOTE_combine_results])

# chart_df = merge_results[merge_results['features']=='depression only']
chart_df = merge_results

line=alt.Chart(chart_df).mark_line(size=1).encode(
    x=alt.X('features',
           sort=['top feat','depression only','top feat+depression']),
    y=alt.Y('recall', title='Recall', scale=alt.Scale(domain=[0, .82])),
    color='model:N'
).properties(
                width=200,
                height=200
                )

points=alt.Chart(chart_df).mark_point().encode(
    x=alt.X('features',
           sort=['top feat','depression only','top feat+depression']),
    y=alt.Y('recall', title='Recall', scale=alt.Scale(domain=[0, .82])),
    color='model:N')


recall = alt.layer(line, points, data=chart_df).facet(row='SMOTE:N',
                                                     column='nulls dropped')
recall

alt.FacetChart(...)

In [35]:
merge_results.sort_values('recall', ascending=False)

,model,precision,recall,f1-score,features,nulls dropped,SMOTE
2,Logistic Regression,0.596383,0.714536,0.598440,top feat+depression,False,True
2,Gaussian Naive Bayes,0.597451,0.714269,0.601114,top feat+depression,False,True
2,Logistic Regression,0.586135,0.699520,0.580058,top feat,False,True
2,Gaussian Naive Bayes,0.626397,0.695179,0.646847,top feat+depression,False,False
2,Logistic Regression,0.592655,0.694240,0.598979,depression only,False,True
...,...,...,...,...,...,...,...
2,Logistic Regression,0.409091,0.409091,0.409091,top feat+depression,True,False
2,Decision Tree,0.409091,0.409091,0.409091,top feat+depression,True,False
2,Knn,0.445455,0.397727,0.390991,top feat+depression,True,True
2,Bernoulli Naive Bayes,0.400000,0.363636,0.380952,top feat+depression,True,False


In [36]:
#All women
merge_results_women=pd.concat([baseline_model_results_women,
                        baseline_model_with_smote_results_women,
                        baseline_model_dropnull_results_women,
                        baseline_model_dropnull_SMOTE_results_women,
                        baseline_model_top_results_women,
                        baseline_model_with_smote_top_results_women,
                        baseline_model_dropnull_top_results_women,
                        baseline_model_dropnull_SMOTE_top_results_women,
                        baseline_model_combine_results_women,
                        baseline_model_with_smote_combine_results_women,
                        baseline_model_dropnull_combine_results_women,
                        baseline_model_dropnull_SMOTE_combine_results_women])

# chart_df = merge_results[merge_results['features']=='depression only']
chart_df = merge_results_women

line=alt.Chart(chart_df).mark_line(size=1).encode(
    x=alt.X('features',
           sort=['top feat','depression only','top feat+depression']),
    y=alt.Y('recall', title='Recall', scale=alt.Scale(domain=[0, .82])),
    color='model:N'
).properties(
                width=200,
                height=200
                )

points=alt.Chart(chart_df).mark_point().encode(
    x=alt.X('features',
           sort=['top feat','depression only','top feat+depression']),
    y=alt.Y('recall', title='Recall', scale=alt.Scale(domain=[0, .82])),
    color='model:N')


recall_women = alt.layer(line, points, data=chart_df).facet(row='SMOTE:N',
                                                     column='nulls dropped')
recall_women

alt.FacetChart(...)

In [37]:
merge_results_women.sort_values('recall', ascending=False)

,model,precision,recall,f1-score,features,nulls dropped,SMOTE
2,Logistic Regression,0.595946,0.803513,0.604621,top feat+depression,False,True
2,Gaussian Naive Bayes,0.595673,0.789997,0.607738,top feat+depression,False,True
2,Logistic Regression,0.582757,0.782564,0.577484,top feat,False,True
2,Gaussian Naive Bayes,0.606959,0.780855,0.631011,top feat+depression,False,False
2,Gaussian Naive Bayes,0.578892,0.773846,0.569747,top feat,False,True
...,...,...,...,...,...,...,...
2,Logistic Regression,0.428571,0.461538,0.444444,top feat+depression,True,False
2,Knn,0.471053,0.435294,0.381887,top feat,True,True
2,Gaussian Naive Bayes,0.462662,0.432353,0.290909,top feat,True,False
2,Decision Tree,0.421875,0.397059,0.409091,top feat,True,False


### Undersampling

Small differences in performance observed between evaluated class imbalance techniques.
RandomOverSampler yields highest recall value for the seven baseline models.

In [38]:
# combined feat, no dropping nulls, yes SMOTE

x_train, x_test, y_train, y_test = get_model_data(
       original_df=cdc_survey_pmom,
       columns = combined_feat,
       test_size_to_use=0.2,
       drop_null_rows=False,
       null_imputer_strategy='median', # mean, median, most_frequent
       use_value_scaler=True,
       use_smote=False)

rus = RandomUnderSampler(
    random_state=42, 
    replacement = True
)
x_rus, y_rus = rus.fit_resample(x_train,y_train)
print(Counter(y_train))
print(Counter(y_rus))

Counter({0: 5599, 1: 593})
Counter({0: 593, 1: 593})


In [39]:
baseline_model_rus = baseline_models(x_rus, y_rus, x_test, y_test, do_smote=False)
baseline_model_rus[['model','precision','recall','f1-score']]


,model,precision,recall,f1-score
2,Gradient Boosting Classifier,0.598676,0.729621,0.595769
2,Random Forest,0.591203,0.722868,0.577240
2,Logistic Regression,0.600816,0.718225,0.606982
2,Gaussian Naive Bayes,0.608829,0.708721,0.624464
2,Bernoulli Naive Bayes,0.584752,0.681836,0.586863
2,Knn,0.574955,0.671578,0.567089
2,Decision Tree,0.555763,0.641804,0.521075


In [40]:
from imblearn.under_sampling import TomekLinks

tomek = TomekLinks()
x_tl, y_tl = tomek.fit_resample(x_train,y_train)
print(Counter(y_train))
print(Counter(y_tl))

Counter({0: 5599, 1: 593})
Counter({0: 5496, 1: 593})


In [41]:
baseline_model_tl = baseline_models(x_tl, y_tl, x_test, y_test, do_smote=False)
baseline_model_tl[['model','precision','recall','f1-score']]


,model,precision,recall,f1-score
2,Gaussian Naive Bayes,0.623993,0.693740,0.644204
2,Bernoulli Naive Bayes,0.619086,0.682241,0.637876
2,Logistic Regression,0.706129,0.577978,0.602657
2,Gradient Boosting Classifier,0.715959,0.567558,0.589939
2,Decision Tree,0.539766,0.540209,0.539984
2,Random Forest,0.721627,0.539709,0.548728
2,Knn,0.614650,0.533594,0.539265


In [42]:
from imblearn.under_sampling import NearMiss

nm = NearMiss()
x_nm, y_nm = nm.fit_resample(x_train, y_train)
print(Counter(y_train))
print(Counter(y_nm))

Counter({0: 5599, 1: 593})
Counter({0: 593, 1: 593})


In [43]:
baseline_model_nm = baseline_models(x_nm, y_nm, x_test, y_test, do_smote=False)
baseline_model_nm[['model','precision','recall','f1-score']]


,model,precision,recall,f1-score
2,Knn,0.565804,0.676813,0.506038
2,Bernoulli Naive Bayes,0.558141,0.657644,0.468082
2,Gradient Boosting Classifier,0.558515,0.648699,0.411199
2,Logistic Regression,0.557136,0.645195,0.409805
2,Random Forest,0.558605,0.644104,0.395232
2,Decision Tree,0.551570,0.623519,0.377516
2,Gaussian Naive Bayes,0.542459,0.598351,0.356842


### Oversampling

In [44]:
ros = RandomOverSampler(random_state=42)
x_ros, y_ros = ros.fit_resample(x_train,y_train)
print(Counter(y_train))
print(Counter(y_ros))

Counter({0: 5599, 1: 593})
Counter({0: 5599, 1: 5599})


In [45]:
baseline_model_ros = baseline_models(x_ros, y_ros, x_test, y_test, do_smote=False)
baseline_model_ros[['model','precision','recall','f1-score']]


,model,precision,recall,f1-score
2,Gradient Boosting Classifier,0.621995,0.739448,0.640479
2,Logistic Regression,0.609616,0.733055,0.619930
2,Gaussian Naive Bayes,0.614652,0.727137,0.630670
2,Bernoulli Naive Bayes,0.583705,0.679051,0.585760
2,Knn,0.579004,0.637299,0.589195
2,Decision Tree,0.585340,0.581994,0.583601
2,Random Forest,0.721350,0.570703,0.594388


In [46]:
baseline_model_with_smote = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
baseline_model_with_smote[['model','precision','recall','f1-score']]

,model,precision,recall,f1-score
2,Logistic Regression,0.596383,0.714536,0.598440
2,Gaussian Naive Bayes,0.597451,0.714269,0.601114
2,Bernoulli Naive Bayes,0.590993,0.682566,0.599614
2,Knn,0.564267,0.649380,0.550992
2,Decision Tree,0.609452,0.608230,0.608835
2,Gradient Boosting Classifier,0.727452,0.599272,0.629458
2,Random Forest,0.725971,0.576632,0.602361


## Segmentation analysis

Lowest recall scores observed in women who have been pregnant population

In [47]:
#All responders
x_train, x_test, y_train, y_test = get_model_data(
    original_df = cdc_survey,
    columns = all_columns
)

baseline_model = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
all_results = baseline_model[['model','precision','recall','f1-score']]
all_results['segment']='all'
all_results

,model,precision,recall,f1-score,segment
2,Logistic Regression,0.573921,0.806275,0.584464,all
2,Gaussian Naive Bayes,0.552218,0.780037,0.526949,all
2,Bernoulli Naive Bayes,0.564390,0.737927,0.575607,all
2,Knn,0.533140,0.628389,0.526244,all
2,Decision Tree,0.580061,0.612031,0.592503,all
2,Gradient Boosting Classifier,0.692383,0.586145,0.616320,all
2,Random Forest,0.711377,0.546940,0.571082,all


In [48]:
#Men only

cdc_survey_men = cdc_survey[cdc_survey['is_male'] == 1]

x_train, x_test, y_train, y_test = get_model_data(
    original_df = cdc_survey_men,
    columns = all_columns
)

baseline_model = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
men_results=baseline_model[['model','precision','recall','f1-score']]
men_results['segment']='men'
men_results

,model,precision,recall,f1-score,segment
2,Logistic Regression,0.564548,0.789287,0.577890,men
2,Gaussian Naive Bayes,0.545979,0.786142,0.526058,men
2,Bernoulli Naive Bayes,0.550562,0.715436,0.558027,men
2,Gradient Boosting Classifier,0.693367,0.613783,0.642175,men
2,Knn,0.525051,0.604450,0.519241,men
2,Decision Tree,0.566877,0.587652,0.575467,men
2,Random Forest,0.711301,0.541248,0.564272,men


In [49]:
#Women only

cdc_survey_women = cdc_survey[cdc_survey['is_male'] == 0]

x_train, x_test, y_train, y_test = get_model_data(
    original_df = cdc_survey_women,
    columns = all_columns
)

baseline_model = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
women_results=baseline_model[['model','precision','recall','f1-score']]
women_results['segment']='women'
women_results

,model,precision,recall,f1-score,segment
2,Logistic Regression,0.592382,0.777401,0.603749,women
2,Gaussian Naive Bayes,0.576862,0.776410,0.561388,women
2,Bernoulli Naive Bayes,0.581812,0.710197,0.596681,women
2,Knn,0.550574,0.636923,0.550879,women
2,Decision Tree,0.598019,0.616457,0.606062,women
2,Gradient Boosting Classifier,0.748938,0.594548,0.630921,women
2,Random Forest,0.795209,0.547121,0.569534,women


In [50]:
#Pregnant women only

x_train, x_test, y_train, y_test = get_model_data(
    original_df = cdc_survey_pmom,
    columns = all_columns
)

baseline_model = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
preg_results=baseline_model[['model','precision','recall','f1-score']]
preg_results['segment']='mothers'
preg_results

,model,precision,recall,f1-score,segment
2,Logistic Regression,0.602697,0.718678,0.610819,mothers
2,Bernoulli Naive Bayes,0.608781,0.689505,0.626282,mothers
2,Gaussian Naive Bayes,0.565029,0.667228,0.529321,mothers
2,Knn,0.539551,0.597337,0.508503,mothers
2,Gradient Boosting Classifier,0.751060,0.586786,0.616888,mothers
2,Decision Tree,0.570494,0.575159,0.572662,mothers
2,Random Forest,0.748243,0.557857,0.577293,mothers


In [51]:
merge_results=pd.concat([all_results, men_results, women_results, preg_results])


line=alt.Chart(merge_results).mark_line(size=1).encode(
    x=alt.X('segment', title = ''),
    y=alt.Y('recall', title='Recall', scale=alt.Scale(domain=[0, .9])),
    color='model:N'
).properties(
                width=200,
                height=200
                )

points=alt.Chart(merge_results).mark_point().encode(
    x=alt.X('segment'),
    y=alt.Y('recall', title='Recall', scale=alt.Scale(domain=[0, .9])),
    color='model:N')


recall = alt.layer(line, points, data=merge_results)
recall

alt.LayerChart(...)